## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

##  Section 1

#### Import required libraries

In [1]:
import numpy as np # library for vectorized computation

import pandas as pd # library to process data as dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # 'requests' offers the most friendly API for opening files, including JSON support
import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

# Install and then import the BeautifulSoup library so we can parse HTML and XML documents
!pip install bs4 
from bs4 import BeautifulSoup

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

#### Scrape Canada postcodes from Wikipedia

In [2]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# open the url and put the html into the soup variable
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html,'html.parser')

#### Load the values in each row into the variables(A, B, C)

In [3]:
# Use the class_ attribute to find the table we want  
all_table=soup.find('table', class_='wikitable sortable')

A=[]
B=[]
C=[]

# Use TR and TD to locate teh rows and cells to load into the A, B and C variables
for row in all_table.findAll('tr'):
    
    cells=row.findAll('td')
    
    if len(cells)==3:  # if there are exactly three TD open / close pairs in a TR row then load data into variables
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

#### Load variables into a dataframe with three columns: PostalCode, Borough, and Neighbourhood

In [4]:
# create a dataframe and load with variables
dfToronto = pd.DataFrame(A,columns=['PostalCode'])
dfToronto['Borough']=B
dfToronto['Neighbourhood']=C

# Use regex to check for the search condition ('\n' is used to create a new line) and then replace with '' 
dfToronto['PostalCode'] = dfToronto['PostalCode'].replace('\n','', regex=True)
dfToronto['Borough'] = dfToronto['Borough'].replace('\n','', regex=True)
dfToronto['Neighbourhood'] = dfToronto['Neighbourhood'].replace('\n','', regex=True)

#### Only include rows with an assigned Borough. Ignore cells with a Borough that is Not assigned

In [5]:
# First, replace Not Assigned values with Numpy's Not a Number and update the dataframe
# Second, use dropna to remove any rows that contain a field with no value

dfToronto['Borough'].replace('Not assigned', np.nan, inplace=True)
dfToronto.dropna(subset=['Borough'], inplace=True)
dfToronto.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Combine rows so that each row has a unique postal code.  Seperate multiple dfTorOnly with a comma

In [6]:
dfToronto.groupby('PostalCode').agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()
dfToronto.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### If a cell has a Borough but a Not assigned Neighbourhood, then the Neighbourhood will be the same as the Borough

In [7]:
print(dfToronto.loc[dfToronto['Neighbourhood'] == 'Not assigned']) # ... this shows that there are no 'Not assigned' values in Neighbourhood

# But in case there were, this function replaces the Neighbourhood not assigned value with the bourough value
dfToronto['Neighbourhood']=dfToronto['Borough'].where(dfToronto['Neighbourhood'].eq('Not assigned'),dfToronto['Neighbourhood'])

dfToronto.head(12)

Empty DataFrame
Columns: [PostalCode, Borough, Neighbourhood]
Index: []


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [8]:
dfToronto.shape

(103, 3)

##  Section 2

#### Load the latitude and longitude coordinates from the csv file downloaded to local machine

In [9]:
dfGeoloc = pd.read_csv('Geospatial_Coordinates.csv')

# rename the postal code field in dfGeoloc to match dfToronto and enable the merge
dfGeoloc.rename(columns={'Postal Code': 'PostalCode'}, inplace=True) 
dfGeoloc.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the two dataframes on the PostalCode field

In [10]:
df_merge = pd.merge(dfToronto, dfGeoloc, on='PostalCode')
df_merge.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Section 3

#### Explore and cluster the dfTorOnly in Toronto, working only with Boroughs that contain the word Toronto

In [45]:
dfTorOnly = df_merge[df_merge.Borough.str.contains('Toronto',case=False)]

#### Restricting the Borough field to cells containing Toronto reduces the size of the dataframe from 103 rows down to 39 rows

In [12]:
dfTorOnly.shape

(39, 5)

#### There are now four unique Boroughs to use for clustering

In [13]:
dfTorOnly.Borough.unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

#### Create a map of Toronto for plotting

In [14]:
!conda install -c conda-forge folium --y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.12.0              |     pyhd8ed1ab_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          64 KB

The following packages will be UPDATED:

  folium                                         0.5.0-py_0 --> 0.12.0-pyhd8ed1ab_0



folium-0.12.0        | 64 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [15]:
import folium # map rendering library

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(dfTorOnly['Latitude'], dfTorOnly['Longitude'], dfTorOnly['Borough'], dfTorOnly['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Solving environment: done

# All requested packages already installed.

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Slice the original dataframe to create seperate dataframes for each of the four boroughs that contain Toronto in their name

In [16]:
Downtown_data = dfTorOnly[dfTorOnly['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Central_data = dfTorOnly[dfTorOnly['Borough'] == 'Central Toronto'].reset_index(drop=True)
East_data = dfTorOnly[dfTorOnly['Borough'] == 'East Toronto'].reset_index(drop=True)
West_data = dfTorOnly[dfTorOnly['Borough'] == 'West Toronto'].reset_index(drop=True)

In [17]:
Central_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678


#### Now get the geolocation for Downtown Toronto

In [18]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="dt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [19]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_data['Latitude'], Downtown_data['Longitude'], Downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [20]:
Downtown_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

#### Need to split out the multiple neighbourhoods in each row of the Downtown Toronto dataframe to enable more granular analysis

In [21]:
# Step 1 - We start with creating a new dataframe from the series with PostalCode as the index
Unique_neighbour = pd.DataFrame(Downtown_data.Neighbourhood.str.split(', ').tolist(), index=Downtown_data.PostalCode).stack() 

# Step 2 - We now want to get rid of the secondary index
# To do this, we will make EmployeeId as a column (it can't be an index since the values will be duplicate)
Unique_neighbour = Unique_neighbour.reset_index([0, 'PostalCode'])

# Step 3 - Set the column names for the new split data as we want them, otherwise Neighbourhood coolumn name shows as 0
Unique_neighbour.columns = ['PostalCode', 'Neighbourhood']

# Add the long and lat coords back into the dataframe
Unique_neighbour = pd.merge(Unique_neighbour, dfGeoloc, on='PostalCode')
Unique_neighbour.head()

,PostalCode,Neighbourhood,Latitude,Longitude
0,M5A,Regent Park,43.654260,-79.360636
1,M5A,Harbourfront,43.654260,-79.360636
2,M7A,Queen's Park,43.662301,-79.389494
3,M7A,Ontario Provincial Government,43.662301,-79.389494
4,M5B,Garden District,43.657162,-79.378937


#### Recreating the Downtown Toronto map to show all neighbourhoods now they're all split out

In [22]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(Unique_neighbour['Latitude'], Unique_neighbour['Longitude'], Unique_neighbour['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

#### Define foursquare credentials and version

In [23]:
CLIENT_ID = 'OCFTHA2C3X3WZMKCDJZAUJLJZTVQ30V0MWLD2IJY3B0EJB02' # your Foursquare ID
CLIENT_SECRET = 'VYHLSBR12TQEMWGFYEH3RAAXFEVJMAKW0ZBZPM3IZ3OA441U' # your Foursquare Secret
ACCESS_TOKEN = 'HNVUYIX22T342L3ZGFY5YAMP5GSFWUY1MCCW1GJOOM0L40MM' # your FourSquare Access Token
VERSION = '20200605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OCFTHA2C3X3WZMKCDJZAUJLJZTVQ30V0MWLD2IJY3B0EJB02
CLIENT_SECRET:VYHLSBR12TQEMWGFYEH3RAAXFEVJMAKW0ZBZPM3IZ3OA441U


#### Get the locations name

In [24]:
Unique_neighbour.loc[0, 'Neighbourhood']

'Regent Park'

In [25]:
neighbourhood_latitude = Unique_neighbour.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = Unique_neighbour.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = Unique_neighbour.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park are 43.6542599, -79.3606359.


In [26]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighbourhood_latitude, neighbourhood_longitude, radius, LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OCFTHA2C3X3WZMKCDJZAUJLJZTVQ30V0MWLD2IJY3B0EJB02&client_secret=VYHLSBR12TQEMWGFYEH3RAAXFEVJMAKW0ZBZPM3IZ3OA441U&v=20200605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [27]:
results = requests.get(url).json()

#### function that extracts the category of the venue

In [28]:
# create new function and pass an argument (row) so that when the function is called, we pass along a row, which is used inside the function to ...
def get_category_type(row):

    try:    # the try block lets us test a block of code for errors
        categories_list = row['categories']    # searches the row for the categories value and stores this in the 'categories_list' variable

    except:    # the except block lets us handle the error
        categories_list = row['venue.categories']    # when the categories value cannot be found in a row, the venue.categories field is stored in the variable
        
    if len(categories_list) == 0:    # if no categories or venue.categories have been stored in teh variable then retun nothing
        return None
    else:
        return categories_list[0]['name']    # If the variable has data then return the name of the 

In [29]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

45 venues were returned by Foursquare.


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
DT_venues = getNearbyVenues(names=Unique_neighbour['Neighbourhood'],
                                   latitudes=Unique_neighbour['Latitude'],
                                   longitudes=Unique_neighbour['Longitude']
                                  )

Regent Park
Harbourfront
Queen's Park
Ontario Provincial Government
Garden District
Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Harbourfront East
Union Station
Toronto Islands
Toronto Dominion Centre
Design Exchange
Commerce Court
Victoria Hotel
University of Toronto
Harbord
Kensington Market
Chinatown
Grange Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
Stn A PO Boxes
St. James Town
Cabbagetown
First Canadian Place
Underground city
Church and Wellesley


In [33]:
print(DT_venues.shape)
DT_venues.head()

(2371, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [34]:
DT_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,93,93,93,93,93,93
Bathurst Quay,18,18,18,18,18,18
Berczy Park,55,55,55,55,55,55
CN Tower,18,18,18,18,18,18
Cabbagetown,43,43,43,43,43,43
Central Bay Street,61,61,61,61,61,61
Chinatown,59,59,59,59,59,59
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79


In [35]:
print('There are {} unique categories'.format(len(DT_venues['Venue Category'].unique())))

There are 206 unique categories


In [36]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
DT_onehot['Neighbourhood'] = DT_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Let examine the dataframe size

In [37]:
DT_onehot.shape

(2371, 207)

#### Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [38]:
DT_grouped = DT_onehot.groupby('Neighbourhood').mean().reset_index()
DT_grouped.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021505,0.0,0.0,0.010753,0.010753,0.0,0.010753,0.0,0.0,0.0,0.000000,0.021505,0.000000,0.010753,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.021505,0.000000,0.010753,0.010753,0.0,0.0,0.010753,0.010753,0.021505,0.000000,0.053763,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.032258,0.000000,0.107527,0.0,0.0,0.0,0.0,0.0,0.010753,0.000000,0.0,0.021505,0.000000,0.021505,0.000000,0.010753,0.0,0.032258,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.010753,0.0,0.000000,0.010753,0.0,0.000000,0.0,0.010753,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.010753,0.0,0.000000,0.0,0.0,0.010753,0.010753,0.000000,0.000000,0.000000,0.032258,0.010753,0.000000,0.0,0.0,0.0,0.0,0.0,0.021505,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.010753,0.010753,0.000000,0.010753,0.0,0.0,0.010753,0.0,0.000000,0.021505,0.000000,0.0,0.010753,0.0,0.0,0.0,0.0,0.010753,0.0,0.0,0.0,0.0,0.000000,0.0,0.010753,0.010753,0.010753,0.010753,0.0,0.010753,0.0,0.0,0.0,0.000000,0.0,0.

#### Lets get the shape of the data

In [39]:
DT_grouped.shape

(38, 207)

#### Print each neighbourhood with the top5 venues

In [40]:
num_top_venues = 5

for hood in DT_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.11
1             Café  0.05
2       Restaurant  0.04
3              Gym  0.03
4  Thai Restaurant  0.03


----Bathurst Quay----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3     Boat or Ferry  0.06
4           Airport  0.06


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----CN Tower----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3     Boat or Ferry  0.06
4           Airport  0.06


----Cabbagetown----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.05
2         Pizza Place  0.05
3                 Pub  0.05
4  Italian Restaurant  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Re

#### Put the data into a dataframe

First, let's write a function to sort the venues in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
print(type(DT_grouped['Neighbourhood']))
print(DT_grouped['Neighbourhood'])

<class 'pandas.core.series.Series'>
0                          Adelaide
1                     Bathurst Quay
2                       Berczy Park
3                          CN Tower
4                       Cabbagetown
5                Central Bay Street
6                         Chinatown
7                          Christie
8              Church and Wellesley
9                    Commerce Court
10                  Design Exchange
11             First Canadian Place
12                  Garden District
13                      Grange Park
14                          Harbord
15                     Harbourfront
16                Harbourfront East
17                Harbourfront West
18                   Island airport
19                Kensington Market
20                             King
21                 King and Spadina
22    Ontario Provincial Government
23                     Queen's Park
24                    Railway Lands
25                      Regent Park
26                         R

Now create the dataframe with the top 10 venues for each neighbourhood 

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = DT_grouped['Neighbourhood']

for ind in np.arange(DT_grouped.shape[0]):    # e.g. for index value in 38 rows
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)
    
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Clothing Store,Pizza Place,Sushi Restaurant,Bakery
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Farmers Market,Restaurant,Juice Bar,Beach
3,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
4,Cabbagetown,Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Bakery,Café,Chinese Restaurant,Grocery Store,Playground


## Cluster neighbourhoods

Run _k_-mean to cluster the neighbourhood into 5 clusters

In [44]:
# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 1, 2, 2, 2, 0, 2, 2], dtype=int32)

Now create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [46]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DT_merged = Unique_neighbour

# merge DT_grouped with manhattan_data to add latitude/longitude for each neighborhood
DT_merged = DT_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

DT_merged.head() # check the last columns!

,PostalCode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Regent Park,43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Restaurant,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop
1,M5A,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Restaurant,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop
2,M7A,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place,Distribution Center
3,M7A,Ontario Provincial Government,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place,Distribution Center
4,M5B,Garden District,43.657162,-79.378937,2,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Ramen Restaurant


Finally, let's visualize the resulting clusters

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'], DT_merged['Longitude'], DT_merged['Neighbourhood'], DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now examine each cluster and determine the discriminating venue categories that distinguish each cluster. Assign a name to each cluster based on the defining categories.

#### Cluster 1: Residential area catering to families

In [50]:
DT_merged.loc[DT_merged['Cluster Labels'] == 0, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop


#### Cluster 2: At the airport

In [55]:
DT_merged.loc[DT_merged['Cluster Labels'] == 1, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
26,King and Spadina,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
27,Railway Lands,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
28,Harbourfront West,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
29,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
30,South Niagara,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina
31,Island airport,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Coffee Shop,Sculpture Garden,Harbor / Marina


#### Cluster 3: Business area catering to office workers

In [52]:
DT_merged.loc[DT_merged['Cluster Labels'] == 2, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park,Coffee Shop,Bakery,Pub,Park,Restaurant,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop
1,Harbourfront,Coffee Shop,Bakery,Pub,Park,Restaurant,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop
2,Queen's Park,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place,Distribution Center
3,Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place,Distribution Center
4,Garden District,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Ramen Restaurant
5,Ryerson,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Ramen Restaurant
6,St. James Town,Coffee Shop,Café,Bakery,Restaurant,Italian Restaurant,Gastropub,Cocktail Bar,Farmers Market,American Restaurant,Japanese Restaurant
7,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Farmers Market,Restaurant,Juice Bar,Beach
8,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Salad Place,Burger Joint,Poke Place,Portuguese Restaurant,Comic Shop
10,Richmond,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Clothing Store,Pizza Place,Sushi Restaurant,Bakery


Cluster 4: Hilly nature part of town

In [53]:
DT_merged.loc[DT_merged['Cluster Labels'] == 3, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Rosedale,Park,Trail,Playground,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


Cluster 5: University area catering to students

In [54]:
DT_merged.loc[DT_merged['Cluster Labels'] == 4, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,University of Toronto,Café,Bakery,Bookstore,Japanese Restaurant,Bar,Yoga Studio,Noodle House,College Arts Building,Comfort Food Restaurant,Dessert Shop
21,Harbord,Café,Bakery,Bookstore,Japanese Restaurant,Bar,Yoga Studio,Noodle House,College Arts Building,Comfort Food Restaurant,Dessert Shop
